In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentence-transformers-dataset/__results__.html
/kaggle/input/sentence-transformers-dataset/__notebook__.ipynb
/kaggle/input/sentence-transformers-dataset/__output__.json
/kaggle/input/sentence-transformers-dataset/custom.css
/kaggle/input/sentence-transformers-dataset/sentence-transformers/index.rst
/kaggle/input/sentence-transformers-dataset/sentence-transformers/NOTICE.txt
/kaggle/input/sentence-transformers-dataset/sentence-transformers/setup.cfg
/kaggle/input/sentence-transformers-dataset/sentence-transformers/LICENSE
/kaggle/input/sentence-transformers-dataset/sentence-transformers/.gitignore
/kaggle/input/sentence-transformers-dataset/sentence-transformers/README.md
/kaggle/input/sentence-transformers-dataset/sentence-transformers/requirements.txt
/kaggle/input/sentence-transformers-dataset/sentence-transformers/setup.py
/kaggle/input/sentence-transformers-dataset/sentence-transformers/tests/test_train_stsb.py
/kaggle/input/sentence-transformers-dataset/sentence-tra

In [2]:
!cp -r ../input/sentence-transformers-dataset/sentence-transformers /tmp/st
!pip install /tmp/st

Processing /tmp/st
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126132 sha256=ec4c97f1af320153d230ca0fb23cbf53f688c1f91525590c9d1588db94db762f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8e8kxruk/wheels/75/66/b4/7e3615e01e698b0515db7fe537d95df69218ed36339b7dfb5d
Successfully built sentence-transformers


In [3]:
import os

import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import gensim
import nltk
import spacy
import sentence_transformers
import re
import spacy
import matplotlib.pyplot as plt
import lightgbm
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression, Ridge

In [4]:
# dir = "../data/"
dir = "/kaggle/input/"
prompts_train = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/prompts_train.csv"))
summaries_train = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/summaries_train.csv"))

prompts_test = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/prompts_test.csv"))
summaries_test = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/summaries_test.csv"))

sample_submission = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/sample_submission.csv"))

In [5]:
def preprocess_general(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t.isalpha()]
    return " ".join(tokens)

stop_words = set(nltk.corpus.stopwords.words("english"))
# lemmatizer = nltk.stem.WordNetLemmatizer()

def remove_stopwords_and_lemmatize(text):
    filtered = []
    for word in nltk.tokenize.word_tokenize(text):
        if word.lower() not in stop_words: 
#             filtered.append(lemmatizer.lemmatize(word))
            filtered.append(word)

    return " ".join(filtered)

In [6]:
summaries_train['cleaned_text'] = summaries_train['text'].apply(preprocess_general)
summaries_train['xcleaned_text'] = summaries_train['cleaned_text'].apply(remove_stopwords_and_lemmatize)

In [7]:
summaries_test['cleaned_text'] = summaries_test['text'].apply(preprocess_general)
summaries_test['xcleaned_text'] = summaries_test['cleaned_text'].apply(remove_stopwords_and_lemmatize)

In [8]:
prompts_train['cleaned_prompt_text'] = prompts_train['prompt_text'].apply(lambda x: remove_stopwords_and_lemmatize(preprocess_general(x)))

In [9]:
prompts_test['cleaned_prompt_text'] = prompts_test['prompt_text'].apply(lambda x: remove_stopwords_and_lemmatize(preprocess_general(x)))

In [10]:
!ls ../input

commonlit-evaluate-student-summaries  sentence-transformers-models
sentence-transformers-dataset


In [11]:
sbert_model = sentence_transformers.SentenceTransformer('../input/sentence-transformers-models/all-MiniLM-L6-v2')

In [12]:
prompts_embeddings = sbert_model.encode(prompts_train['cleaned_prompt_text'])
prompts_emb_df = pd.DataFrame(prompts_embeddings, columns=[f"PE_{i}" for i in range(prompts_embeddings.shape[1])], index=prompts_train['prompt_id'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
prompts_embeddings_test = sbert_model.encode(prompts_test['cleaned_prompt_text'])
prompts_emb_df_test = pd.DataFrame(prompts_embeddings_test, columns=[f"PE_{i}" for i in range(prompts_embeddings_test.shape[1])], index=prompts_test['prompt_id'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
summaries_embeddings = sbert_model.encode(summaries_train['xcleaned_text'])
summaries_emb_df = pd.DataFrame(summaries_embeddings, columns=[f"SE_{i}" for i in range(summaries_embeddings.shape[1])])

Batches:   0%|          | 0/224 [00:00<?, ?it/s]

In [15]:
summaries_embeddings_test = sbert_model.encode(summaries_test['xcleaned_text'])
summaries_emb_df_test = \
pd.DataFrame(summaries_embeddings_test, columns=[f"SE_{i}" for i in range(summaries_embeddings_test.shape[1])])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
def similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [17]:
prompt_idx_mapper = dict(zip(prompts_train['prompt_id'].to_list(), prompts_train.index))
summaries_train['prompt_idx'] = summaries_train['prompt_id'].map(prompt_idx_mapper)
summaries_train['summary_idx'] = summaries_train.index
summaries_train['similarity_to_prompt'] = \
    summaries_train.apply(lambda x: similarity(prompts_embeddings[x['prompt_idx']], summaries_embeddings[x['summary_idx']]), axis=1)
summaries_train = summaries_train.drop(columns=['prompt_idx', 'summary_idx'])

In [18]:
prompt_idx_mapper = dict(zip(prompts_test['prompt_id'].to_list(), prompts_test.index))
summaries_test['prompt_idx'] = summaries_test['prompt_id'].map(prompt_idx_mapper)
summaries_test['summary_idx'] = summaries_test.index
summaries_test['similarity_to_prompt'] = \
    summaries_test.apply(lambda x: similarity(prompts_embeddings[x['prompt_idx']], summaries_embeddings[x['summary_idx']]), axis=1)
summaries_test = summaries_test.drop(columns=['prompt_idx', 'summary_idx'])

In [19]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4))
tfidf.fit(summaries_train['cleaned_text'])

TfidfVectorizer(ngram_range=(1, 4))

In [20]:
summaries_train['n_words'] = summaries_train['cleaned_text'].apply(lambda x: len(x.split(" ")))
summaries_train['n_sentences'] = summaries_train['text'].str.count('\.')
summaries_train['mean_word_length'] = summaries_train['cleaned_text'].apply(lambda x: np.mean([len(word) for word in x.split()]))
summaries_train['n_punctuation'] = summaries_train.apply(lambda x: len(x['text']) - len(x['cleaned_text']), axis=1)

In [21]:
summaries_test['n_words'] = summaries_test['cleaned_text'].apply(lambda x: len(x.split(" ")))
summaries_test['n_sentences'] = summaries_test['text'].str.count('\.')
summaries_test['mean_word_length'] = summaries_test['cleaned_text'].apply(lambda x: np.mean([len(word) for word in x.split()]))
summaries_test['n_punctuation'] = summaries_test.apply(lambda x: len(x['text']) - len(x['cleaned_text']), axis=1)

In [22]:
df_content = pd.concat([
    summaries_train[["similarity_to_prompt", "n_words", "n_sentences", "mean_word_length", "n_punctuation", "content"]], 
    prompts_emb_df.loc[summaries_train['prompt_id']].reset_index(drop=True),
    summaries_emb_df
], axis=1, join='inner') 

In [23]:
df_content_test = pd.concat([
    summaries_test[["similarity_to_prompt", "n_words", "n_sentences", "mean_word_length", "n_punctuation"]], 
    prompts_emb_df_test.loc[summaries_test['prompt_id']].reset_index(drop=True),
    summaries_emb_df_test
], axis=1, join='inner') 

In [24]:
predictors = df_content.drop("content", axis=1).columns
target = "content"

In [25]:
lm = Ridge(alpha=10869)
lm.fit(df_content[predictors], df_content[target])

Ridge(alpha=10869)

In [26]:
content_pred = lm.predict(df_content_test[predictors])

In [27]:
df_wording = summaries_train[["n_words", "n_sentences", "mean_word_length", "n_punctuation", "wording", "cleaned_text"]]
df_wording_test = summaries_test[["n_words", "n_sentences", "mean_word_length", "n_punctuation", "cleaned_text"]]

In [28]:
predictors = df_wording.drop(["wording", "cleaned_text"], axis=1).columns
target = 'wording'

In [29]:
X_wording_train = scipy.sparse.hstack([df_wording[predictors], tfidf.transform(df_wording['cleaned_text'])]) 
X_wording_test = scipy.sparse.hstack([df_wording_test[predictors], tfidf.transform(df_wording_test['cleaned_text'])]) 

In [30]:
lm = Ridge(alpha=1346)
lm.fit(X_wording_train, df_wording[target])

Ridge(alpha=1346)

In [31]:
wording_pred = lm.predict(X_wording_test)

In [32]:
sample_submission

,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


In [33]:
sample_submission['content'] = content_pred
sample_submission['wording'] = wording_pred

In [34]:
sample_submission

,student_id,content,wording
0,000000ffffff,-1.134886,-0.720735
1,111111eeeeee,-1.135409,-0.720735
2,222222cccccc,-1.133369,-0.720735
3,333333dddddd,-1.131172,-0.720735


In [35]:
sample_submission.to_csv("submission.csv", index=False)